In [ ]:
from ultralytics import YOLO
import numpy as np
import os

# Load a model
model = YOLO('runs/pose/train50/weights/best.pt')  # load a trained model

# Path to the folder containing images
input_folder = 'yolo_new/4/test4/images'

# Create an output folder for the text files
output_folder = 'yolo_new/4/predicted folder'
os.makedirs(output_folder, exist_ok=True)

# List all image files in the folder and sort them
image_files = sorted([f for f in os.listdir(input_folder) if f.endswith('.png')])

# Iterate through the image files
for image_file in image_files:
    # Predict with the model
    results = model(os.path.join(input_folder, image_file))  # predict on an image

    # Create a flat list to store XY values for the current image
    xy_values = []

    # Iterate through the results
    for result in results:
        if result.keypoints is not None:
            # Extract XY values from the keypoints
            for xy in result.keypoints[0].xy[0].cpu().numpy():  
                xy_values.extend(xy)

    # Create a separate text file for each image
    output_file_path = os.path.join(output_folder, f"{os.path.splitext(image_file)[0]}.txt")

    # Save all XY values to the text file in the same line
    with open(output_file_path, 'w') as file:
        file.write(" ".join([f"{val:.6f}" for val in xy_values]))

print("XY values for all images have been saved to separate text files in the 'output' folder.")


In [ ]:
import cv2
import os

# Paths to folders
image_folder = "yolo_new/4/test4/images/"
ground_truth_folder = "yolo_new/4/test_folder/"
predicted_folder = "yolo_new/4/predicted folder/"

# Create the 'display' folder if it doesn't exist
output_folder = "yolo_new/4/display"
os.makedirs(output_folder, exist_ok=True)

# Get a list of image files in ascending order
image_files = sorted([f for f in os.listdir(image_folder) if f.endswith('.png')])

def read_keypoints(file_path):
    keypoints = []
    with open(file_path, "r") as file:
        lines = file.readlines()
        for line in lines:
            values = line.strip().split()
            # Ensure there are at least two values before processing
            if len(values) >= 2:
                # Process the values as keypoint coordinates
                keypoints.append([(int(float(values[i])), int(float(values[i + 1]))) for i in range(0, min(len(values), len(values) - 1), 2)])
            else:
                # Handle cases where there are not enough values
                print(f"Skipping line in {file_path}: {line.strip()}")

    return keypoints

for i, image_file in enumerate(image_files):
    # Load the image
    image = cv2.imread(os.path.join(image_folder, image_file))

    # Read the predicted and ground truth keypoints
    predicted_file = os.path.join(predicted_folder, image_file.replace('.png', '.txt'))
    ground_truth_file = os.path.join(ground_truth_folder, image_file.replace('.png', '.txt'))

    predicted_keypoints = read_keypoints(predicted_file)
    ground_truth_keypoints = read_keypoints(ground_truth_file)

    # Draw keypoints on the image
    def draw_keypoints(img, keypoints, color, shape):
        for kp in keypoints:
            for x, y in kp:
                if shape == 'circle':
                    cv2.circle(img, (x, y), 3, color, -1)
                elif shape == 'rectangle':
                    cv2.rectangle(img, (x - 3, y - 3), (x + 3, y + 3), color, -1)

    # Draw predicted keypoints in red circles
    draw_keypoints(image, predicted_keypoints, (0, 0, 255), 'circle')

    # Draw ground truth keypoints in blue rectangles
    draw_keypoints(image, ground_truth_keypoints, (0, 255, 0), 'rectangle')

    # Add a label at the bottom of the image with a smaller font size and yellow color
    label = "Predicted (Red Circles) / Ground Truth (Green Rectangles)"
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.4  # Adjust the font size here
    font_thickness = 1
    font_color = (0, 255, 255)  # Yellow color (BGR format)
    text_size = cv2.getTextSize(label, font, font_scale, font_thickness)[0]
    text_x = (image.shape[1] - text_size[0]) // 2  # Centered horizontally
    text_y = image.shape[0] - 10  # Positioned at the bottom
    cv2.putText(image, label, (text_x, text_y), font, font_scale, font_color, font_thickness, cv2.LINE_AA)

    # Save the image with drawn keypoints and labels
    output_image_path = os.path.join(output_folder, image_file)
    cv2.imwrite(output_image_path, image)

print(f"Images with keypoints and labels at the bottom have been saved to the '{output_folder}' folder.")
